from google.colab import drive
drive.mount('/content/drive')

In [31]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import mean_squared_error,mean_absolute_error


In [32]:
Dataset_path = "./DataSets/"
Movies_metadata = pd.read_csv(Dataset_path+"MoviesInfo.csv")
ratings = pd.read_csv(Dataset_path+"ml-latest/ratings.csv")

In [33]:
Movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movieId                41896 non-null  int64  
 1   genres                 41896 non-null  object 
 2   imdb_title_id          41896 non-null  object 
 3   title                  41896 non-null  object 
 4   original_title         41896 non-null  object 
 5   year                   41896 non-null  int64  
 6   date_published         41896 non-null  object 
 7   genre                  41896 non-null  object 
 8   duration               41896 non-null  int64  
 9   country                41890 non-null  object 
 10  language               41648 non-null  object 
 11  director               41886 non-null  object 
 12  writer                 41621 non-null  object 
 13  production_company     40944 non-null  object 
 14  actors                 41877 non-null  object 
 15  de

In [34]:
obj_Movies_metadata = Movies_metadata.select_dtypes(include=['object']).copy()
obj_Movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   genres                 41896 non-null  object
 1   imdb_title_id          41896 non-null  object
 2   title                  41896 non-null  object
 3   original_title         41896 non-null  object
 4   date_published         41896 non-null  object
 5   genre                  41896 non-null  object
 6   country                41890 non-null  object
 7   language               41648 non-null  object
 8   director               41886 non-null  object
 9   writer                 41621 non-null  object
 10  production_company     40944 non-null  object
 11  actors                 41877 non-null  object
 12  description            41589 non-null  object
 13  budget                 15453 non-null  object
 14  usa_gross_income       13291 non-null  object
 15  worlwide_gross_inco

In [35]:
myobj = ['country','language','director','production_company']

for i in myobj:
    obj_Movies_metadata[i] = obj_Movies_metadata[i].astype('category')
    Movies_metadata[i] = obj_Movies_metadata[i].cat.codes


In [36]:
Movies_metadata = Movies_metadata.fillna(100000)
Movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movieId                41896 non-null  int64  
 1   genres                 41896 non-null  object 
 2   imdb_title_id          41896 non-null  object 
 3   title                  41896 non-null  object 
 4   original_title         41896 non-null  object 
 5   year                   41896 non-null  int64  
 6   date_published         41896 non-null  object 
 7   genre                  41896 non-null  object 
 8   duration               41896 non-null  int64  
 9   country                41896 non-null  int16  
 10  language               41896 non-null  int16  
 11  director               41896 non-null  int16  
 12  writer                 41896 non-null  object 
 13  production_company     41896 non-null  int16  
 14  actors                 41896 non-null  object 
 15  de

In [77]:
data =  Movies_metadata[['movieId','year','duration','metascore','reviews_from_users','reviews_from_critics'
                              ,'country','language','director','production_company','mean_vote']]

In [81]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform(data.drop(['movieId','mean_vote'],axis=1))
data_norm = pd.DataFrame(scaled,columns=(data.drop(['movieId','mean_vote'],axis=1).columns))
data_norm= data_norm.join(data['mean_vote'])
data_norm= data_norm.join(data['movieId'])


from scipy import stats
z_scores = stats.zscore(data_norm.drop(['movieId','mean_vote'],axis=1))
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
newdata = data_norm[filtered_entries]

newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38725 entries, 0 to 41894
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  38725 non-null  float64
 1   duration              38725 non-null  float64
 2   metascore             38725 non-null  float64
 3   reviews_from_users    38725 non-null  float64
 4   reviews_from_critics  38725 non-null  float64
 5   country               38725 non-null  float64
 6   language              38725 non-null  float64
 7   director              38725 non-null  float64
 8   production_company    38725 non-null  float64
 9   mean_vote             38725 non-null  float64
 10  movieId               38725 non-null  int64  
dtypes: float64(10), int64(1)
memory usage: 3.5 MB


### Without Classing of target

In [82]:
target = newdata['mean_vote'] 
predictors = newdata.drop(['mean_vote','movieId'],axis=1)

In [83]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30, random_state=40)


In [84]:
model01 = Sequential()
model01.add(Dense(900, activation='relu', input_dim=9))
model01.add(Dense(300, activation='tanh'))
model01.add(Dense(150, activation='relu'))
model01.add(Dense(30, activation='relu'))
model01.add(Dense(1, activation='linear'))

# Compile the model
model01.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])

In [42]:
model01.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
2711/2711 [==============================] - 9s 3ms/step - loss: 0.7445 - mean_squared_error: 1.0156
Epoch 2/20
2711/2711 [==============================] - 9s 3ms/step - loss: 0.7025 - mean_squared_error: 0.8500
Epoch 3/20
2711/2711 [==============================] - 8s 3ms/step - loss: 0.6913 - mean_squared_error: 0.8259
Epoch 4/20
2711/2711 [==============================] - 8s 3ms/step - loss: 0.6868 - mean_squared_error: 0.8204
Epoch 5/20
2711/2711 [==============================] - 14s 5ms/step - loss: 0.6807 - mean_squared_error: 0.8067
Epoch 6/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6755 - mean_squared_error: 0.7997
Epoch 7/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6748 - mean_squared_error: 0.7977
Epoch 8/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6704 - mean_squared_error: 0.7860
Epoch 9/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6678 - mean_squared_e

In [43]:
y_pred= model01.predict(X_test)

In [44]:
scores = model01.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores[1], 1 - scores[1]))    

Accuracy on test data: 0.7804288864135742% 
 Error on test data: 0.21957111358642578


In [45]:
print('MSE:',mean_squared_error(y_test, y_pred))
print('MAE:',mean_absolute_error(y_test, y_pred))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred)))

MSE: 0.780428922065367
MAE: 0.6621373496956086
RMSE: 0.8834188825610232


### With Classing mean_vote

In [85]:
newdata['mean_vote_class'] = 0
newdata['mean_vote_class'][newdata['mean_vote']>3.5] = 1
newdata['mean_vote_class'][newdata['mean_vote']<=3.5] = 0

<ipython-input-85-31771ccae61c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['mean_vote_class'] = 0
<ipython-input-85-31771ccae61c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['mean_vote_class'][newdata['mean_vote']>3.5] = 1
C:\Users\PJ\anaconda3\lib\site-packages\pandas\core\generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inpl

In [86]:
target = newdata['mean_vote_class'] 
predictors = newdata.drop(['mean_vote','mean_vote_class','movieId'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30, random_state=40)


In [87]:
model02 = Sequential()
model02.add(Dense(900, activation='relu', input_dim=9))
model02.add(Dense(300, activation='tanh'))
model02.add(Dense(150, activation='relu'))
model02.add(Dense(30, activation='relu'))
model02.add(Dense(1, activation='linear'))

# Compile the model
model02.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])

In [49]:
model02.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
2711/2711 [==============================] - 18s 6ms/step - loss: 0.0379 - mean_squared_error: 0.0174
Epoch 2/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0205 - mean_squared_error: 0.0149 0s - loss: 0.0206 - mean_squared_error
Epoch 3/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0193 - mean_squared_error: 0.0149
Epoch 4/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0189 - mean_squared_error: 0.0148
Epoch 5/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0184 - mean_squared_error: 0.0148
Epoch 6/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0181 - mean_squared_error: 0.0148
Epoch 7/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0176 - mean_squared_error: 0.0148
Epoch 8/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0173 - mean_squared_error: 0.0148
Epoch 9/20
2711/2711 [==============================] - 1

In [50]:
y_pred= model02.predict(X_test)

In [51]:
scores = model02.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores[1], 1 - scores[1])) 

Accuracy on test data: 0.015060998499393463% 
 Error on test data: 0.9849390015006065


In [52]:
print('MSE:',mean_squared_error(y_test, y_pred))
print('MAE:',mean_absolute_error(y_test, y_pred))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred)))

MSE: 0.015061011789444134
MAE: 0.01529279788342005
RMSE: 0.12272331396048647


In [123]:
import difflib
import random

def generate_recommendation(model,data ,thresh=4):
    
    """
    Generates a Movie recommendation for a user based on a rating threshold. Only
    Movies with a predicted rating at or above the threshold will be recommended
    """
    
    Movies_ID = data.movieId.unique().tolist()
    random.shuffle(Movies_ID)

    for Movie_ID in Movies_ID:
        mask = data[data.movieId.isin([Movie_ID])]
        mask.drop(['movieId'],axis=1)
        rating = model.predict(mask)
        if rating >= thresh:
            return Movie_ID ,title, rating

In [125]:
generate_recommendation(model01,newdata.drop(['mean_vote','mean_vote_class'],axis=1))

ValueError: in user code:

    C:\Users\PJ\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1569 predict_function  *
        return step_function(self, iterator)
    C:\Users\PJ\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\PJ\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\PJ\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\PJ\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\PJ\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1552 run_step  **
        output